# RNN 사용하기

쉽지는 않지만, RNN을 사용해보기로 하자.

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('C:/Users/USER/Desktop/playground/chess_play/chess_rating_pred_dataset.csv')

앞에서 같은 등급의 체스 경기, 블리츠 300초 경기에 한정, 최대 70수 까지만 보고 레이팅을 맞히기로 결정하였다.
해당 데이터 셋을 바탕으로 예측을 해보자.

앞에서 했던 가중치나 유사도 기반의 문제점은
1) 순서에 대한 정보 무시
2) 체스 수라는 것은 상황에 따라 가중치가 다른데, 단순히 수가 많이 등장했다고 가중치를 부과할 수 없음

따라서 순서를 반영하는 RNN을 사용해야 한다. 여기서 수의 중요도보다는 수 자체를 사용해야 하므로, 벡터화 방법도 달라져야 한다.

## 학습 데이터 만들기, 벡터화 방법

우선 가장 큰 문제는 리스트가 지금 문자열로 저장되어 버렸다. 이를 해소하자

In [3]:
df['chess_string'] = df.chess_string.apply(lambda x: eval(x))

그리고, word2vec을 해야 할 것이다.

In [4]:
from gensim.models.word2vec import Word2Vec

In [5]:
from sklearn.model_selection import train_test_split
X = df['chess_string']
y = df['grade']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 42, stratify = y)

window는 4로 했다. 너- 나 - 너- 나 이렇게 4수를 보면 대응까지를 고려하므로 4수가 적당하다.

In [6]:
model = Word2Vec(sentences=X_train, size=10, window=4, min_count=0, sg=0)

In [7]:
model.wv.most_similar("e4")

[('d4', 0.9681786298751831),
 ('Nce7', 0.9479725956916809),
 ('Nf3', 0.9387083649635315),
 ('Nc3', 0.9037632346153259),
 ('e5', 0.8928003907203674),
 ('Bc4', 0.8917235136032104),
 ('d3', 0.8751618266105652),
 ('Nf6', 0.8522761464118958),
 ('d5', 0.8508899211883545),
 ('e6', 0.8503772020339966)]

상당히 유의미한 결과가보인다. 물론 완벽하진 않지만 e4의 가장 유사한 수로 d4와 Nf3를 뽑았다는 것만으로도, w2v의 위력이 보인다.

In [8]:
model['e4']

<ipython-input-8-b771cad6c9b1>:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  model['e4']


array([ 0.63632035,  0.585608  , -0.15454559,  0.82870847, -1.1557916 ,
        1.1156863 , -6.4622827 , -1.2580993 , -0.69398916, -1.6452929 ],
      dtype=float32)

In [9]:
model.wv.most_similar("O-O-O")

[('O-O', 0.9590904712677002),
 ('Nbd7', 0.8902510404586792),
 ('h3', 0.8853350877761841),
 ('Ne7', 0.8754733204841614),
 ('Bg5', 0.8745657205581665),
 ('Nbd2', 0.8723241090774536),
 ('Ngf6', 0.8618440628051758),
 ('Nge2', 0.8575639724731445),
 ('Qd2', 0.8548845052719116),
 ('Be7', 0.8544213771820068)]

캐슬링에 대해서도 캐슬링 길을열거나 킹사이드/퀸사이드의 유사성을 보여주고 있어서 학습이 잘 된 것처럼 보인다.

train에 없는 수가 등장할 수도 있는데, 이는 일관적으로 통일하자.

In [10]:
def get_word2vec_seq(x):
    answer_list = []
    for c in x:
        try:
            answer_list.append(model[c])
        except:
            answer_list.append([0.5] * 10)
    if len(answer_list) < 70:
        while len(answer_list) < 70:
            answer_list.append([0] * 10)
    return answer_list

In [11]:
X_train_w2v = X_train.apply(lambda x : get_word2vec_seq(x))
X_test_w2v = X_test.apply(lambda x : get_word2vec_seq(x))

<ipython-input-10-32b71a9b334d>:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  answer_list.append(model[c])


In [12]:
X_train_w2v

2761    [[-2.3914945, 2.4819613, 1.1999704, 1.04758, -...
5099    [[0.63632035, 0.585608, -0.15454559, 0.8287084...
2322    [[0.63632035, 0.585608, -0.15454559, 0.8287084...
3118    [[0.63632035, 0.585608, -0.15454559, 0.8287084...
2203    [[0.63632035, 0.585608, -0.15454559, 0.8287084...
                              ...                        
94      [[0.63632035, 0.585608, -0.15454559, 0.8287084...
4104    [[0.63632035, 0.585608, -0.15454559, 0.8287084...
6002    [[-0.3204533, 1.0069542, -0.45866463, 1.566056...
7380    [[-0.325715, 0.9255762, 1.1859684, 1.9020668, ...
6281    [[-0.3204533, 1.0069542, -0.45866463, 1.566056...
Name: chess_string, Length: 6473, dtype: object

## pytorch로 rnn 학습하기

In [14]:
import torch
import torch.nn as nn